##### Bibliotecas

In [13]:
import pandas as pd
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

##### Funções

In [14]:
def extrair_partidas(url, data):
    service = Service() # Usada para iniciar uma instância do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized') # Define a preferência de browser para o Chrome
    driver = webdriver.Chrome(service=service, options=options) # Inicia a instância
    time.sleep(5)
    driver.get(url)

    time.sleep(10)
    partidas = driver.find_elements(By.CLASS_NAME, 'tournament-matches-row')
    lista_partidas = []

    # Pega dados
    for i in range(len(partidas)):
        partida = {}
        partida['data'] = data
        partida['horario'] = partidas[i].text.split('\n')[1]
        partida['time_casa'] = partidas[i].text.split('\n')[4]
        partida['player_casa'] = partidas[i].text.split('\n')[5]
        partida['gols_casa'] = int(partidas[i].text.split('\n')[6])
        partida['gols_fora'] = int(partidas[i].text.split('\n')[8])
        if len(partidas[i].text.split('\n')[11]) == 1:
            partida['player_fora'] = partidas[i].text.split('\n')[10] # 10 11
            partida['time_fora'] = partidas[i].text.split('\n')[9] # 9 10
        else:
            partida['player_fora'] = partidas[i].text.split('\n')[11] # 10 11
            partida['time_fora'] = partidas[i].text.split('\n')[10] # 9 10
        partida['total_gols'] = partida['gols_casa'] + partida['gols_fora']
        
        lista_partidas.append(partida)
        
    driver.close()
    driver.quit()
    return lista_partidas

def extrair_torneios(url):
    service = Service() # Usada para iniciar uma instância do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized') # Define a preferência de browser para o Chrome
    driver = webdriver.Chrome(service=service, options=options) # Inicia a instância
    driver.get(url)
    time.sleep(10)
    links = driver.find_elements(By.CLASS_NAME, 'icon-link')

    lista_torneios = []
    for i in range(len(links)):
        lista_torneios.append(links[i].get_attribute('href'))

    return lista_torneios

def salvar(dados):
    dados = dados
    for i in range(len(dados)):
        with open('base_predict.csv', 'a', newline='', encoding='utf-8') as arquivo:
            escrever = csv.writer(arquivo)
            escrever.writerow([dados[i]['data'], dados[i]['horario'], dados[i]['time_casa'], dados[i]['player_casa'], 
            dados[i]['gols_casa'], dados[i]['gols_fora'], dados[i]['player_fora'], dados[i]['time_fora'], 
            dados[i]['total_gols']])
    


In [15]:
# service = Service() # Usada para iniciar uma instância do Chrome
# options = webdriver.ChromeOptions()
# options.add_argument('--start-maximized') # Define a preferência de browser para o Chrome
# driver = webdriver.Chrome(service=service, options=options) # Inicia a instância
# time.sleep(5)
# driver.get('https://football.esportsbattle.com/en/tournament/124037')

# time.sleep(10)
# part = driver.find_elements(By.CLASS_NAME, 'tournament-matches-row')


In [16]:
# # for i in range(len(part)):
# #     print(part[i].text.split('\n')[4])
# #     print(part[i].text.split('\n')[10])
# #     print('----------------')

# print(part[6].text.split('\n')[4])
# print(part[6].text.split('\n')[9])

##### Preparação

In [17]:
# Cria uma lista de datas
list_datas = [(datetime(2024, 2, 21) + timedelta(days=n)).strftime('%Y-%m-%d') 
              for n in range((datetime(2024, 2, 21) - datetime(2024, 2, 21)).days + 1)]

# Cria um lista de URLs com as datas
pags_urls = []
for i in range(len(list_datas)):
    url = f"https://football.esportsbattle.com/en/schedule?page=1&dateFrom={list_datas[i].split('-')[0]}%2F{list_datas[i].split('-')[1]}%2F{list_datas[i].split('-')[2]}+08%3A00&dateTo={list_datas[i].split('-')[0]}%2F{list_datas[i].split('-')[1]}%2F{list_datas[i].split('-')[2]}+23%3A59"
    pags_urls.append(url)    

##### Inicialização das Funções

In [18]:
cont = 0
for url in pags_urls:
    torneios = extrair_torneios(url)
    for torneio in torneios:
        partidas = extrair_partidas(torneio, list_datas[cont])
        salvar(partidas)
    cont += 1
time.sleep(10)    